In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams["font.sans-serif"] = "SimHei" #解决中文乱码问题
import seaborn as sns
import random
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn import model_selection

from sklearn.neighbors import KNeighborsRegressor

In [2]:
import json
# 读取json文件内容,返回字典格式
with open('./filesPath.json','r',encoding='utf8')as fp:
    json_data = json.load(fp)
print(json_data['df_train'])

E:\Visual_Studio_Code_Files\python\competitionData\Tmall\data_format1\train_format1.csv


In [3]:
df_train = pd.read_csv(json_data['df_train'])
df_test = pd.read_csv(json_data['df_test'])
user_info = pd.read_csv(json_data['user_info'])
user_log = pd.read_csv(json_data['user_log'])

In [4]:
df_train.head()

,user_id,merchant_id,label
0,34176,3906,0
1,34176,121,0
2,34176,4356,1
3,34176,2217,0
4,230784,4818,0


In [5]:
print(df_test.shape,df_train.shape)
print(user_info.shape,user_log.shape)

(261477, 3) (260864, 3)
(424170, 3) (54925330, 7)


In [6]:
user_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424170 entries, 0 to 424169
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   user_id    424170 non-null  int64  
 1   age_range  421953 non-null  float64
 2   gender     417734 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 9.7 MB


In [7]:
user_info.head(10)

,user_id,age_range,gender
0,376517,6.0,1.0
1,234512,5.0,0.0
2,344532,5.0,0.0
3,186135,5.0,0.0
4,30230,5.0,0.0
5,272389,6.0,1.0
6,281071,4.0,0.0
7,139859,7.0,0.0
8,198411,5.0,1.0
9,67037,4.0,1.0


In [9]:
user_info.drop_duplicates(subset='age_range', keep='first', inplace=False)

,user_id,age_range,gender
0,376517,6.0,1.0
1,234512,5.0,0.0
6,281071,4.0,0.0
7,139859,7.0,0.0
17,349112,3.0,1.0
28,171799,0.0,1.0
263,328576,8.0,1.0
715,316419,2.0,0.0
10983,315780,NaN,NaN
52194,106650,1.0,0.0


In [10]:
user_info.drop_duplicates(subset='gender', keep='first', inplace=False)

,user_id,age_range,gender
0,376517,6.0,1.0
1,234512,5.0,0.0
10,149002,5.0,2.0
348,169941,4.0,NaN


In [11]:
user_info['age_range'].replace(0.0,np.nan,inplace=True)
user_info['gender'].replace(2.0,np.nan,inplace=True)
user_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424170 entries, 0 to 424169
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   user_id    424170 non-null  int64  
 1   age_range  329039 non-null  float64
 2   gender     407308 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 9.7 MB


In [12]:
user_info['age_range'].replace(np.nan,-1,inplace=True)
user_info['gender'].replace(np.nan,-1,inplace=True)
user_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424170 entries, 0 to 424169
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   user_id    424170 non-null  int64  
 1   age_range  424170 non-null  float64
 2   gender     424170 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 9.7 MB
